In [2]:
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
import numpy as np
import scipy.linalg as la
from qiskit_aer import AerSimulator
from qiskit.circuit import Gate
from matplotlib import pyplot as plt
import time
import os

In [156]:
def create_a(N_max: int) -> np.ndarray:
    a = np.zeros((N_max,) * 2)
    for i in range(N_max - 1):
        a[i, i + 1] = np.sqrt(i + 1)
    a_dag = a.T
    return a, a_dag

def create_H(N_max: int, lam: float) -> np.ndarray:
    a, a_dag = create_a(N_max)
    x = (a + a_dag) / np.sqrt(2)
    x4 = x @ x @ x @ x
    return a_dag @ a + 0.5 * np.eye(N_max) + lam * x4

def create_x(N_max: int) -> np.ndarray:
    a, a_dag = create_a(N_max)
    return (a + a_dag) / np.sqrt(2)

The matrix representation of $x^2$ and $x^4$ in the SHO basis are
\begin{equation}
x^2 =
\begin{pmatrix}
\frac{1}{2} & 0 & \frac{1}{\sqrt{2}} & 0 \\
0 & \frac{3}{2} & 0 & \sqrt{\frac{3}{2}} \\
\frac{1}{\sqrt{2}} & 0 & \frac{5}{2} & 0 \\
0 & \sqrt{\frac{3}{2}} & 0 & \frac{3}{2}
\end{pmatrix}
\end{equation}

\begin{equation}
x^4 = \begin{pmatrix} \frac{3}{4} & 0 & \frac{3}{\sqrt{2}} & 0 \\
0 & \frac{15}{4} & 0 & 3 \sqrt{\frac{3}{2}} \\
\frac{3}{\sqrt{2}} & 0 & \frac{27}{4} & 0 \\
0 & 3\sqrt{\frac{3}{2}} & 0 & \frac{15}{4}
\end{pmatrix}
\end{equation}

In [258]:
x_mat = create_x(N_max)

In [259]:
x2_mat = x_mat @ x_mat

In [260]:
x2_op = SparsePauliOp.from_operator(x2_mat)

In [160]:
x4_mat = x2_mat @ x2_mat

In [161]:
x4_op = SparsePauliOp.from_operator(x4_mat)

In [162]:
x4_coeffs = x4_op.coeffs.real

In [163]:
x2_coeffs = x2_op.coeffs.real

In [261]:
x2_coeffs

array([ 1.5       ,  0.96592583, -0.25881905, -0.5       , -0.5       ])

In [262]:
x2_op.paulis

PauliList(['II', 'XI', 'XZ', 'ZI', 'ZZ'])

In [263]:
x4_op.paulis

PauliList(['II', 'XI', 'XZ', 'ZI', 'ZZ'])

In [167]:
SparsePauliOp.from_operator(create_H(4, 0.0))

SparsePauliOp(['II', 'IZ', 'ZI'],
              coeffs=[ 2. +0.j, -0.5+0.j, -1. +0.j])

In [269]:
def exp_adia(qc, lam, m_ctr, deltat, strang=False):
    if strang:
        qc.rz(- 2 * 0.5 * deltat / 2, 0)
        qc.rz(- 2 * 1 * deltat / 2, 1)
        qc.rz(2 * (lam * x4_coeffs[3] + m_ctr * x2_coeffs[3]) * deltat / 2, 1)
        qc.rzz(2 * (lam * x4_coeffs[4] + m_ctr * x2_coeffs[4]) * deltat / 2, 1, 0)
        qc.rx(2 * (lam * x4_coeffs[1] + m_ctr * x2_coeffs[1]) * deltat, 1)
        qc.h(1)
        qc.rzz(2 * (lam * x4_coeffs[2] + m_ctr * x2_coeffs[2]) * deltat, 1, 0)
        qc.h(1)
        qc.rz(2 * (lam * x4_coeffs[3] + m_ctr * x2_coeffs[3]) * deltat / 2, 1)
        qc.rzz(2 * (lam * x4_coeffs[4] + m_ctr * x2_coeffs[4]) * deltat / 2, 1, 0)
        qc.rz(- 2 * 0.5 * deltat / 2, 0)
        qc.rz(- 2 * 1 * deltat / 2, 1)
        # qc.global_phase -= deltat * lam * x4_coeffs[0] + deltat * m_ctr * x2_coeffs[0] + 2 * deltat
    else:
        qc.rx(2 * (lam * x4_coeffs[1] + m_ctr * x2_coeffs[1]) * deltat, 1)
        qc.h(1)
        qc.rzz(2 * (lam * x4_coeffs[2] + m_ctr * x2_coeffs[2]) * deltat, 1, 0)
        qc.h(1)
        qc.rz(2 * (lam * x4_coeffs[3] + m_ctr * x2_coeffs[3]) * deltat, 1)
        qc.rzz(2 * (lam * x4_coeffs[4] + m_ctr * x2_coeffs[4]) * deltat, 1, 0)
        qc.rz(- 2 * 0.5 * deltat, 0)
        qc.rz(- 2 * 1 * deltat, 1)
        # qc.global_phase -= deltat * lam * x4_coeffs[0] + deltat * m_ctr * x2_coeffs[0] + 2 * deltat

In [270]:
def r(s):
    return s * s * (3  - 2 * s)

In [271]:
def lamOf(lam, u):
    if u > 1:
        return lam
    return lam * r(u)

def dm2Of(lam, u):
    if u > 1:
        return - 6 * lam * (1 - r(u - 1))
    return - 6 * lam * r(u)

In [272]:
def create_adiabatic(lam, t_prep, n_steps, strang=False):
    qc = QuantumCircuit(2)
    dt = t_prep / (2 * n_steps)
    for j in range(1, 2 * n_steps + 1):
        u = (j - 0.5) / n_steps
        exp_adia(qc, lamOf(lam, u), dm2Of(lam, u) / 2, dt, strang)
    return qc

In [312]:
qc = create_adiabatic(0.5, 10, 20, strang=False)

In [313]:
from qiskit.quantum_info import Statevector

In [314]:
state = Statevector.from_instruction(qc)

In [315]:
np_state = np.linalg.eigh(create_H(4, 0.5)).eigenvectors[0]

In [316]:
np.abs(np.vdot(state, np_state))**2

np.float64(0.8230139513372297)